In [4]:
import pandas as pd
import plotly.graph_objects as go

In [5]:
# df is your output dataframe with columns:
# 'trade_date_year_mo', 'delivery_year_mo', 'weighted_avg_price', 'total_transacted_quantity'
# use 2019 as test
df = pd.read_csv(r"/Users/ankitakhatri/Downloads/STA 160/EQR_output_EDA.csv")

In [6]:
# Ensure datetime types
df['trade_date_year_mo'] = pd.to_datetime(df['trade_date_year_mo'])
df['delivery_year_mo']   = pd.to_datetime(df['delivery_year_mo'])

# Get unique delivery months as strings
month_labels_all = sorted(df['delivery_year_mo'].dt.strftime('%Y-%m').unique())

def process_month_data(month_str):
    """
    Filter df to a single delivery month and apply:
    - sort by trade date
    - expand full monthly range
    - interpolate weighted_avg_price
    - fill missing quantity with 0
    Returns a DataFrame indexed by trade_date_year_mo, or None if empty.
    """
    mask = df['delivery_year_mo'].dt.strftime('%Y-%m') == month_str
    g = df.loc[mask].copy()
    if g.empty:
        return None

    # Sort by trade date
    g = g.sort_values('trade_date_year_mo')

    # Full month range to avoid breaks
    full_range = pd.date_range(
        start=g['trade_date_year_mo'].min(),
        end=g['trade_date_year_mo'].max(),
        freq='MS'  # Month Start
    )

    g = g.set_index('trade_date_year_mo').reindex(full_range)
    g.index.name = 'trade_date_year_mo'

    # Interpolate / fill
    g['weighted_avg_price'] = g['weighted_avg_price'].interpolate()
    g['total_transacted_quantity'] = g['total_transacted_quantity'].fillna(0)

    return g

# --- 2. Build frames: one frame per delivery month ---

frames = []
valid_month_labels = []   # only months that actually have data

for month_str in month_labels_all:
    g = process_month_data(month_str)
    if g is None:
        continue

    valid_month_labels.append(month_str)

    line_trace = go.Scatter(
        x=g.index,
        y=g['weighted_avg_price'],
        mode='lines',
        name='Weighted Avg Price',
        yaxis='y1'
    )

    bar_trace = go.Bar(
        x=g.index,
        y=g['total_transacted_quantity'],
        name='Total Transacted Quantity',
        opacity=0.4,
        yaxis='y2'
    )

    # Dynamic axes in each frame: autorange = True
    frames.append(
        go.Frame(
            name=month_str,
            data=[line_trace, bar_trace],
            layout=go.Layout(
                title_text=f"Delivery Month: {month_str}",
                xaxis=dict(
                    autorange=True,
                    tickformat="%Y-%m"
                ),
                yaxis=dict(
                    autorange=True
                ),
                yaxis2=dict(
                    autorange=True
                )
            )
        )
    )

if not frames:
    raise ValueError("No frames were created – check your data.")

initial_frame = frames[0]

# --- 3. Slider that selects frames by delivery month ---

slider_steps = []
for month_str in valid_month_labels:
    slider_steps.append({
        "label": month_str,
        "method": "animate",
        "args": [
            [month_str],   # go to the frame with this name
            {
                "mode": "immediate",
                "frame": {"duration": 0, "redraw": True},
                "transition": {"duration": 0}
            }
        ],
    })

sliders = [{
    "active": 0,
    "pad": {"t": 50},
    "x": 0.1,
    "y": -0.1,
    "len": 0.8,
    "steps": slider_steps
}]

# Optional play button
updatemenus = [{
    "type": "buttons",
    "showactive": False,
    "x": 0.1,
    "y": 1.12,
    "buttons": [
        {
            "label": "Play",
            "method": "animate",
            "args": [
                None,
                {
                    "frame": {"duration": 700, "redraw": True},
                    "fromcurrent": True,
                    "transition": {"duration": 0}
                }
            ]
        }
    ]
}]

# --- 4. Create the figure with dual y-axes and dynamic autorange ---

fig = go.Figure(
    data=initial_frame.data,
    layout=go.Layout(
        title=initial_frame.layout.title,
        xaxis=dict(
            title="Trade Date (YYYY-MM)",
            tickformat="%Y-%m",
            autorange=True
        ),
        yaxis=dict(
            title="Weighted Avg Price ($/KW-MO)",
            autorange=True
        ),
        yaxis2=dict(
            title="Total Transacted Quantity (MW)",
            overlaying="y",
            side="right",
            autorange=True
        ),
        sliders=sliders,
        updatemenus=updatemenus,
        margin=dict(l=70, r=70, t=90, b=110),  # extra bottom margin so slider isn't cut off
    ),
    frames=frames
)

# --- 5. Show it in a notebook / browser ---

fig.show()

# --- 6. (Optional) Export as standalone interactive HTML file ---

fig.write_html("interactive_price_by_delivery_month.html")
print("Saved interactive chart to interactive_price_by_delivery_month.html")


Saved interactive chart to interactive_price_by_delivery_month.html
